<a href="https://colab.research.google.com/github/candido05/Repositorio-do-livro-Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-Tensorflow-/blob/main/Capitulo11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capítulo 11 – Treinando Redes Neurais Profundas

In [1]:
import tensorflow as tf
import numpy as np

### Problema de Gradiente de Fuga e Explosão de Gradiente

* Inicialização de Xavier e He

In [2]:
dense = tf.keras.layers.Dense(50, activation='relu', kernel_initializer='he_normal')

In [3]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_avg',
                                                    distribution='uniform')
desne = tf.keras.layers.Dense(50, activation='sigmoid', kernel_initializer=he_avg_init)

* Funções de ativação de não saturação

In [4]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Dense(10, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(alpha=0.2)
])

In [5]:
# PReLU

model = tf.keras.models.Sequential([

    tf.keras.layers.Dense(10, kernel_initializer="he_normal"),
    tf.keras.layers.PReLU()
])

In [6]:
# SELU

layer = tf.keras.layers.Dense(10, activation="selu",
                              kernel_initializer="lecun_normal")

### Normalização em batch

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [8]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (Batch  (None, 784)               3136      
 Normalization)                                                  
                                                                 
 dense_5 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Bat  (None, 100)              

In [9]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [10]:
model.layers[1].updates

<ipython-input-10-11fe563bf3e2>:1: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model.layers[1].updates


[]

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

### Gradient Clipping

In [12]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

### Otimização Momentum

In [13]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

### Gradiente acelerado de Nesterov

In [14]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9,
                                    nesterov=True)

### AdaGrad

In [15]:
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)

### RMSProp

In [16]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

### Otimização Adam e Nadam

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9,
                                     beta_2=0.999)

In [18]:
optimizer = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9,
                                       beta_2=0.999)

In [19]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9,
                                      beta_2=0.999)

In [20]:
optimizer = tf.keras.optimizers.AdamW(weight_decay=1e-5, learning_rate=0.001,
                                      beta_1=0.9, beta_2=0.999)

### Evitando Sobreajuste na Regularização

* regularização _L1_ e _L2_

In [21]:
layer = tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [22]:
from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense,
                           activation="relu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

* Dropout

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

* Regularização Max-Norm

In [24]:
dense = tf.keras.layers.Dense(
    100, activation="relu", kernel_initializer="he_normal",
    kernel_constraint=tf.keras.constraints.max_norm(1.))

## Exercícios

### Q. 8

* _a)_

In [25]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
  model.add(tf.keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))

* _b)_

In [26]:
#output layer

model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [27]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer, metrics=["accuracy"])

In [28]:
cifar10 = tf.keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

print(X_train_full.shape)
print(y_train_full.shape)
print(X_test.shape)
print(y_test.shape)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [29]:
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [30]:
from pathlib import Path

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model", save_best_only=True)

callbacks = [early_stopping_cb, model_checkpoint_cb]

In [31]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 32s 14ms/step - loss: 4.9723 - accuracy: 0.1608 - val_loss: 2.1691 - val_accuracy: 0.2170
Epoch 2/100
1407/1407 [==============================] - 18s 13ms/step - loss: 2.0846 - accuracy: 0.2445 - val_loss: 2.0976 - val_accuracy: 0.2306
Epoch 3/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.9651 - accuracy: 0.2806 - val_loss: 1.9275 - val_accuracy: 0.2912
Epoch 4/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.8843 - accuracy: 0.3100 - val_loss: 1.8603 - val_accuracy: 0.3324
Epoch 5/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.8219 - accuracy: 0.3335 - val_loss: 1.8270 - val_accuracy: 0.3322
Epoch 6/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.7772 - accuracy: 0.3569 - val_loss: 1.7577 - val_accuracy: 0.3614
Epoch 7/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.7345 - accuracy: 0.3704 - val_loss: 1

In [32]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 3ms/step - loss: 1.5265 - accuracy: 0.4668


[1.5265058279037476, 0.4668000042438507]

* _c)_

In [33]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
  model.add(tf.keras.layers.Dense(100, kernel_initializer='he_normal'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('elu'))

model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [34]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer, metrics=["accuracy"])

In [35]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_ckeckpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_bn_model",
                                                         save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]

In [36]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 63s 29ms/step - loss: 1.8413 - accuracy: 0.3413 - val_loss: 1.7298 - val_accuracy: 0.3760
Epoch 2/100
1407/1407 [==============================] - 41s 29ms/step - loss: 1.6651 - accuracy: 0.4064 - val_loss: 1.7104 - val_accuracy: 0.3860
Epoch 3/100
1407/1407 [==============================] - 42s 30ms/step - loss: 1.5953 - accuracy: 0.4322 - val_loss: 1.6463 - val_accuracy: 0.4234
Epoch 4/100
1407/1407 [==============================] - 40s 29ms/step - loss: 1.5484 - accuracy: 0.4496 - val_loss: 1.7288 - val_accuracy: 0.3844
Epoch 5/100
1407/1407 [==============================] - 39s 28ms/step - loss: 1.5060 - accuracy: 0.4654 - val_loss: 1.5604 - val_accuracy: 0.4346
Epoch 6/100
1407/1407 [==============================] - 40s 28ms/step - loss: 1.4706 - accuracy: 0.4775 - val_loss: 1.6920 - val_accuracy: 0.4008
Epoch 7/100
1407/1407 [==============================] - 41s 29ms/step - loss: 1.4367 - accuracy: 0.4908 - val_loss: 1

In [37]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 1s 4ms/step - loss: 1.4003 - accuracy: 0.5220


[1.4002516269683838, 0.5220000147819519]

* _d)_

In [38]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100,
                                    kernel_initializer="lecun_normal",
                                    activation="selu"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

optimizer = tf.keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=20, restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "my_cifar10_selu_model", save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 27s 13ms/step - loss: 1.9195 - accuracy: 0.3148 - val_loss: 1.7859 - val_accuracy: 0.3714
Epoch 2/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.6985 - accuracy: 0.3996 - val_loss: 1.6930 - val_accuracy: 0.3994
Epoch 3/100
1407/1407 [==============================] - 18s 12ms/step - loss: 1.6095 - accuracy: 0.4280 - val_loss: 1.6023 - val_accuracy: 0.4336
Epoch 4/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.5423 - accuracy: 0.4544 - val_loss: 1.5726 - val_accuracy: 0.4590
Epoch 5/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.4898 - accuracy: 0.4757 - val_loss: 1.5619 - val_accuracy: 0.4504
Epoch 6/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.4406 - accuracy: 0.4955 - val_loss: 1.5267 - val_accuracy: 0.4650
Epoch 7/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.3926 - accuracy: 0.5155 - val_loss: 1

[1.4682574272155762, 0.5019999742507935]